In [51]:
stack = 'MD595'
list_fn = '/home/yuncong/CSHL_data_processed/' + stack + '_filename_map.txt'

with open(list_fn, 'r') as f:
    fns = [l.split()[0] for l in f.readlines()]
    
with open(list_fn, 'w') as f:
    for i, fn in enumerate(fns):
        f.write(fn + ' ' + str(i+1) + '\n')

In [14]:
with open('/home/yuncong/Downloads/kleinfeldMD602.txt', 'r') as f:
    valid = [l.rstrip() for l in f.readlines() if l != '\n']    

In [19]:
set(fns) - set(valid)

{'MD602-IHC26-2015.12.01-23.19.04_MD602_2_0077',
 'MD602-IHC28-2015.12.01-23.27.06_MD602_2_0083',
 'MD602-IHC32-2015.12.01-23.43.41_MD602_1_0096',
 'MD602-IHC38-2015.12.02-00.12.04_MD602_1_0114',
 'MD602-N27-2015.12.01-17.05.22_MD602_1_0081',
 'MD602-N34-2015.12.01-17.36.30_MD602_1_0102'}

In [20]:
set(valid) - set(fns)

{'MD602-IHC15-2015.12.01-22.30.50_MD602_1_0045',
 'MD602-N15-2015.12.01-16.04.47_MD602_2_0044'}

In [18]:
[fn for fn in fns if fn in valid]

['MD602-IHC1-2015.12.01-21.34.39_MD602_1_0001',
 'MD602-N1-2015.12.01-15.03.14_MD602_1_0001',
 'MD602-IHC1-2015.12.01-21.34.39_MD602_2_0002',
 'MD602-N1-2015.12.01-15.03.14_MD602_2_0002',
 'MD602-IHC1-2015.12.01-21.34.39_MD602_3_0003',
 'MD602-IHC2-2015.12.01-21.36.53_MD602_1_0004',
 'MD602-IHC2-2015.12.01-21.36.53_MD602_2_0005',
 'MD602-IHC2-2015.12.01-21.36.53_MD602_3_0006',
 'MD602-N3-2015.12.01-15.08.09_MD602_1_0007',
 'MD602-IHC3-2015.12.01-21.39.48_MD602_1_0007',
 'MD602-N3-2015.12.01-15.08.09_MD602_2_0008',
 'MD602-IHC3-2015.12.01-21.39.48_MD602_2_0008',
 'MD602-N3-2015.12.01-15.08.09_MD602_3_0009',
 'MD602-IHC3-2015.12.01-21.39.48_MD602_3_0009',
 'MD602-N4-2015.12.01-15.11.54_MD602_1_0010',
 'MD602-IHC4-2015.12.01-21.43.41_MD602_1_0010',
 'MD602-N4-2015.12.01-15.11.54_MD602_2_0011',
 'MD602-IHC4-2015.12.01-21.43.41_MD602_2_0011',
 'MD602-IHC4-2015.12.01-21.43.41_MD602_1_0012',
 'MD602-N4-2015.12.01-15.11.54_MD602_3_0012',
 'MD602-N5-2015.12.01-15.15.39_MD602_1_0013',
 'MD602-IH

In [30]:
import numpy as np

In [43]:
def parameter_file_to_dict(filename):
	d = {}
	with open(filename, 'r') as f:
		for line in f.readlines():
			if line.startswith('('):
				tokens = line[1:-2].split(' ')
				key = tokens[0]
				if len(tokens) > 2:
					value = []
					for v in tokens[1:]:
						try:
							value.append(float(v))
						except ValueError:
							value.append(v)
				else:
					v = tokens[1]
					try:
						value = (float(v))
					except ValueError:
						value = v
				d[key] = value

		return d

def parse_parameter_file(filepath):

	d = parameter_file_to_dict(filepath)

	rot_rad, x_mm, y_mm = d['TransformParameters']
	center = np.array(d['CenterOfRotationPoint']) / np.array(d['Spacing'])
	# center[1] = d['Size'][1] - center[1]

	xshift = x_mm / d['Spacing'][0]
	yshift = y_mm / d['Spacing'][1]

	R = np.array([[np.cos(rot_rad), -np.sin(rot_rad)],
	              [np.sin(rot_rad), np.cos(rot_rad)]])
	shift = center + (xshift, yshift) - np.dot(R, center)
	T = np.vstack([np.column_stack([R, shift]), [0,0,1]])

	return T

In [48]:
# q = parse_parameter_file('/home/yuncong/elastix_output/TransformParameters.0.txt')
q = parse_parameter_file('/home/yuncong/TransformParameters_33_32.txt')

In [50]:
q

array([[   1.,   -0.,   65.],
       [   0.,    1.,  105.],
       [   0.,    0.,    1.]])

In [49]:
np.linalg.inv(q)

array([[   1.,    0.,  -65.],
       [   0.,    1., -105.],
       [   0.,    0.,    1.]])